<a href="https://colab.research.google.com/github/xvilaysouk/NTL_DMFA/blob/main/NTL_DMFA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>